# Historical and future simulation of hailstorms in Australia

In [1]:
%cd git/hist_future_hail/

/home/561/tr2908/git/hist_future_hail


In [2]:
import modules.hist_fut_hail as hf
from importlib import reload
import numpy as np

## Settings

In [3]:
base_dir = '/g/data/up6/tr2908/hist_future_hail/'   # Data base directory.
hist_years = np.arange(1989,2009)                   # Historical years.
fut_years = np.arange(2079,2099)                    # Future years.
wrf_template_dir = f'{base_dir}/WRF_v4.4/template/' # Template directory for WRF/WPS setups.
sims_dir = f'{base_dir}/WRF_v4.4/simulations'       # Simulation directory.

## Input data

Write scripts that can be run to download all the boundary condition data.

WRF boundary conditions are supplied from [Xu et al., 2021](https://www.nature.com/articles/s41597-021-01079-3) (data are CC-BY-4.0):

- Publication: https://www.nature.com/articles/s41597-021-01079-3
- Data archive: https://www.scidb.cn/en/detail?dataSetId=791587189614968832

In [ ]:
# hf.gen_download_script(years=hist_years, out_file=f'{base_dir}/xu_data/hist/download_hist.sh')
# hf.gen_download_script(years=fut_years, out_file=f'{base_dir}/xu_data/ssp245/download_hist.sh')

## WRF setup

In [8]:
reload(hf)

<module 'modules.hist_fut_hail' from '/home/561/tr2908/git/hist_future_hail/modules/hist_fut_hail.py'>

In [10]:
hf.set_up_WRF(year=1989, template_dir=wrf_template_dir, sims_dir=sims_dir, exp='hist')

Skipping existing WPS...


In [ ]:
import xarray

In [ ]:
foo = xarray.open_dataset('/g/data/w42/tr2908/pv_hailswaths/mesh/radar_57_annual_max_mesh.nc')

In [ ]:
foo.isel(year=5).mesh.plot()